### 2022 Data

In [1]:
import pandas as pd
import numpy as np

In [60]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022" # fetching all the 2022 movies from wikipedia
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [61]:
df = pd.concat([df1,df2,df3,df4],ignore_index=True) #concat all the dataframes in a single for preprocessing
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2]
1,J A N U A R Y,7.0,The Legend of La Llorona,Saban Films / Ageless Pictures,Patricia Harris Seeley (director); José Prende...,[3]
2,J A N U A R Y,7.0,The Commando,Saban Films / Premiere Entertainment,Asif Akbar (director); Koji Steven Sakai (scre...,[4]
3,J A N U A R Y,7.0,American Siege,Vertical Entertainment,Edward John Drake (director/screenplay); Timot...,[5]
4,J A N U A R Y,14.0,Scream,Paramount Pictures / Spyglass Media Group / Ra...,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[6]
...,...,...,...,...,...,...
316,D E C E M B E R,30.0,"Alice, Darling",Lionsgate / Elevation Pictures,Mary Nighy (director); Alanna Francis (screenp...,[265]
317,D E C E M B E R,NaN,NaN,NaN,NaN,NaN
318,D E C E M B E R,NaN,NaN,NaN,NaN,NaN
319,D E C E M B E R,NaN,NaN,NaN,NaN,NaN


In [62]:
df.columns

Index(['Opening', 'Opening.1', 'Title', 'Production company', 'Cast and crew',
       'Ref.'],
      dtype='object')

#### As it is observed in the above table, we do not have the genre column so we will get the genre data from IMDb. You will need to create an API key in order to access the API key.

### pip install tmdbv3api

In [63]:
#get your API Key here: https://www.themoviedb.org/settings/api
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()

api = 'cf1e5b9d8cf31e19913dd6bc256abb2a'
tmdb.api_key=api # insert your api key here


In [64]:
import requests

from tmdbv3api import Movie

tmdb_movie = Movie()

def get_genre(x):
    genres = []
    
    try:
        # Search for the movie title
        result = tmdb_movie.search(x)
        
        # Check if any results were found
        if not result:
            return 'No results found'

        # Get the movie ID from the first result
        movie_id = result[0].id
        
        # Fetch movie details using the movie ID
        response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}')
        data_json = response.json()
        
        # Check if the movie has genres
        if 'genres' in data_json and data_json['genres']:
            for genre in data_json['genres']:
                genres.append(genre['name'])
            return ' '.join(genres)  # Return genres as a space-separated string
        else:
            return np.nan
    
    except Exception as e:
        # Handle any exceptions (e.g., connection issues, API errors)
        return f'Error: {str(e)}'

# Example usage to map genres to the DataFrame
df['genres'] = df['Title'].map(lambda x: get_genre(x))


In [65]:
# df['genres']= df['Title'].map(lambda x : get_genre(x))

df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.,genres
0,J A N U A R Y,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2],Action Adventure Thriller
1,J A N U A R Y,7.0,The Legend of La Llorona,Saban Films / Ageless Pictures,Patricia Harris Seeley (director); José Prende...,[3],Family Animation Fantasy Horror
2,J A N U A R Y,7.0,The Commando,Saban Films / Premiere Entertainment,Asif Akbar (director); Koji Steven Sakai (scre...,[4],Action Crime Thriller
3,J A N U A R Y,7.0,American Siege,Vertical Entertainment,Edward John Drake (director/screenplay); Timot...,[5],Action Adventure Thriller
4,J A N U A R Y,14.0,Scream,Paramount Pictures / Spyglass Media Group / Ra...,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[6],Horror Mystery
...,...,...,...,...,...,...,...
316,D E C E M B E R,30.0,"Alice, Darling",Lionsgate / Elevation Pictures,Mary Nighy (director); Alanna Francis (screenp...,[265],Thriller Drama Romance
317,D E C E M B E R,NaN,NaN,NaN,NaN,NaN,Error: quote_from_bytes() expected bytes
318,D E C E M B E R,NaN,NaN,NaN,NaN,NaN,Error: quote_from_bytes() expected bytes
319,D E C E M B E R,NaN,NaN,NaN,NaN,NaN,Error: quote_from_bytes() expected bytes


In [66]:
import re
import pandas as pd

def extract_info2(text):
    # Check if the text is None or not a string
    if not isinstance(text, str):
        return pd.Series([None, None, None, None])
    
    # Initialize the director to None
    director_name = None
    # Initialize the actors to None
    actors = [None, None, None]
    
    # Check if the text contains a director's name and role(s)
    director_match = re.search(r'([a-zA-Z\s\.\-]+)\s*\(([^)]+)\)', text)  # Extract name and roles inside parentheses
    if director_match:
        name = director_match.group(1).strip()  # Extract the name (e.g., Michael Moore)
        roles = director_match.group(2).strip().lower()  # Extract the roles (e.g., director/screenplay/narrator)
        
        # If 'director' is in the roles, assign only the name to the director column
        if 'director' in roles:
            director_name = name  # Only assign the name to the director column if 'director' is present in the roles
            
            # Remove the director's name from the text to avoid duplication
            text = text.replace(director_match.group(0), "").strip()  # Remove the entire director's entry

    # Extract actors from the remaining text (after the last semicolon, if any)
    actors_section = re.split(r';', text)[-1].strip()  # Split by semicolon and take the last part
    actors_list = [actor.strip() for actor in actors_section.split(',') if actor.strip()]  # Split by comma and clean up
    
    # Remove the director from the actors list (if it was listed there)
    actors_list = [actor for actor in actors_list if actor != director_name]
    
    # Assign up to 3 actors
    for i in range(min(len(actors_list), 3)):  # Ensure we don't exceed 3 actors
        actors[i] = actors_list[i]

    # Return the extracted information in a pandas Series
    return pd.Series([director_name, actors[0], actors[1], actors[2]])

In [67]:
df[['director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name']] = df['Cast and crew'].apply(extract_info2)


In [68]:
df=df.rename(columns={'Title':'movie_title'})
df.head(2)

,Opening,Opening.1,movie_title,Production company,Cast and crew,Ref.,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,J A N U A R Y,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2],Action Adventure Thriller,Simon Kinberg,Jessica Chastain,Penélope Cruz,Fan Bingbing
1,J A N U A R Y,7.0,The Legend of La Llorona,Saban Films / Ageless Pictures,Patricia Harris Seeley (director); José Prende...,[3],Family Animation Fantasy Horror,Patricia Harris Seeley,Autumn Reeser,Danny Trejo,Antonio Cupo


In [89]:
new_df22 = df[['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Simon Kinberg,Jessica Chastain,Penélope Cruz,Fan Bingbing,Action Adventure Thriller,The 355
1,Patricia Harris Seeley,Autumn Reeser,Danny Trejo,Antonio Cupo,Family Animation Fantasy Horror,The Legend of La Llorona
2,Asif Akbar,Mickey Rourke,Michael Jai White,None,Action Crime Thriller,The Commando
3,Edward John Drake,Timothy V. Murphy,Bruce Willis,Rob Gough,Action Adventure Thriller,American Siege
4,Tyler Gillett,Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery,Scream
...,...,...,...,...,...,...
316,Mary Nighy,Anna Kendrick,Kaniehtiio Horn,Charlie Carrick,Thriller Drama Romance,"Alice, Darling"
317,None,None,None,None,Error: quote_from_bytes() expected bytes,NaN
318,None,None,None,None,Error: quote_from_bytes() expected bytes,NaN
319,None,None,None,None,Error: quote_from_bytes() expected bytes,NaN


In [90]:
new_df22.shape

(321, 6)

In [91]:
new_df22.isna().sum()

director_name     4
actor_1_name      5
actor_2_name     13
actor_3_name     34
genres            2
movie_title       4
dtype: int64

In [92]:
new_df22 = new_df22.iloc[:-4]
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Simon Kinberg,Jessica Chastain,Penélope Cruz,Fan Bingbing,Action Adventure Thriller,The 355
1,Patricia Harris Seeley,Autumn Reeser,Danny Trejo,Antonio Cupo,Family Animation Fantasy Horror,The Legend of La Llorona
2,Asif Akbar,Mickey Rourke,Michael Jai White,None,Action Crime Thriller,The Commando
3,Edward John Drake,Timothy V. Murphy,Bruce Willis,Rob Gough,Action Adventure Thriller,American Siege
4,Tyler Gillett,Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery,Scream
...,...,...,...,...,...,...
312,Kasi Lemmons,Naomi Ackie,Stanley Tucci,Ashton Sanders,Music History Drama,Whitney Houston: I Wanna Dance with Somebody
313,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Thriller Crime Horror Mystery,The Pale Blue Eye
314,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,Drama,Women Talking
315,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Comedy Drama,A Man Called Otto


In [93]:
new_df22.dropna(thresh=1,inplace=True)

C:\Users\ARMS\AppData\Local\Temp\ipykernel_5732\2519640007.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df22.dropna(thresh=1,inplace=True)


In [94]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Simon Kinberg,Jessica Chastain,Penélope Cruz,Fan Bingbing,Action Adventure Thriller,The 355
1,Patricia Harris Seeley,Autumn Reeser,Danny Trejo,Antonio Cupo,Family Animation Fantasy Horror,The Legend of La Llorona
2,Asif Akbar,Mickey Rourke,Michael Jai White,None,Action Crime Thriller,The Commando
3,Edward John Drake,Timothy V. Murphy,Bruce Willis,Rob Gough,Action Adventure Thriller,American Siege
4,Tyler Gillett,Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery,Scream
...,...,...,...,...,...,...
312,Kasi Lemmons,Naomi Ackie,Stanley Tucci,Ashton Sanders,Music History Drama,Whitney Houston: I Wanna Dance with Somebody
313,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Thriller Crime Horror Mystery,The Pale Blue Eye
314,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,Drama,Women Talking
315,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Comedy Drama,A Man Called Otto


In [95]:
new_df22.fillna('unknown',inplace=True)

C:\Users\ARMS\AppData\Local\Temp\ipykernel_5732\589004298.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df22.fillna('unknown',inplace=True)


In [96]:
new_df22.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
dtype: int64

In [97]:
new_df22['movie_title']= new_df22['movie_title'].str.lower()

C:\Users\ARMS\AppData\Local\Temp\ipykernel_5732\4239953512.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df22['movie_title']= new_df22['movie_title'].str.lower()


In [98]:
new_df22[new_df22['actor_3_name']=='unknown']

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
2,Asif Akbar,Mickey Rourke,Michael Jai White,unknown,Action Crime Thriller,the commando
7,Rick Jacobson,Laura Marano,Mena Massoud,unknown,Romance Comedy,the royal treatment
15,Paul Solet,Adrien Brody,unknown,unknown,Crime Action Drama,clean
28,Arie Posin,Joey King,Kyle Allen,unknown,Romance Science Fiction Drama,the in between
51,Adrian Lyne,Ben Affleck,Ana de Armas,unknown,Drama Mystery Thriller,deep water
62,Roshan Sethi,Karan Soni,Geraldine Viswanathan,unknown,Crime Horror Drama Mystery Thriller,7 days
80,Jeff Wilson,Catherine Keener (narrator),unknown,unknown,Documentary Family,polar bear
85,Reilly,Bruce Willis,Michael Rooker,unknown,Science Fiction Action Thriller Adventure,corrective measures
95,Chadd Harbold,Ashley Benson,Shiloh Fernandez,unknown,Drama Thriller,private property
102,Richard Wong,Eugenio Derbez,Samara Weaving,unknown,Comedy Romance,the valet


In [99]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Simon Kinberg,Jessica Chastain,Penélope Cruz,Fan Bingbing,Action Adventure Thriller,the 355
1,Patricia Harris Seeley,Autumn Reeser,Danny Trejo,Antonio Cupo,Family Animation Fantasy Horror,the legend of la llorona
2,Asif Akbar,Mickey Rourke,Michael Jai White,unknown,Action Crime Thriller,the commando
3,Edward John Drake,Timothy V. Murphy,Bruce Willis,Rob Gough,Action Adventure Thriller,american siege
4,Tyler Gillett,Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery,scream
...,...,...,...,...,...,...
312,Kasi Lemmons,Naomi Ackie,Stanley Tucci,Ashton Sanders,Music History Drama,whitney houston: i wanna dance with somebody
313,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Thriller Crime Horror Mystery,the pale blue eye
314,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,Drama,women talking
315,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Comedy Drama,a man called otto


In [100]:

new_df22['comb'] = new_df22['actor_1_name'] + ' ' + new_df22['actor_2_name'] + ' '+ new_df22['actor_3_name'] + ' '+ new_df22['director_name'] +' ' + new_df22['genres']
new_df22

C:\Users\ARMS\AppData\Local\Temp\ipykernel_5732\1696911016.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df22['comb'] = new_df22['actor_1_name'] + ' ' + new_df22['actor_2_name'] + ' '+ new_df22['actor_3_name'] + ' '+ new_df22['director_name'] +' ' + new_df22['genres']


,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Simon Kinberg,Jessica Chastain,Penélope Cruz,Fan Bingbing,Action Adventure Thriller,the 355,Jessica Chastain Penélope Cruz Fan Bingbing Si...
1,Patricia Harris Seeley,Autumn Reeser,Danny Trejo,Antonio Cupo,Family Animation Fantasy Horror,the legend of la llorona,Autumn Reeser Danny Trejo Antonio Cupo Patrici...
2,Asif Akbar,Mickey Rourke,Michael Jai White,unknown,Action Crime Thriller,the commando,Mickey Rourke Michael Jai White unknown Asif A...
3,Edward John Drake,Timothy V. Murphy,Bruce Willis,Rob Gough,Action Adventure Thriller,american siege,Timothy V. Murphy Bruce Willis Rob Gough Edwar...
4,Tyler Gillett,Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery,scream,Melissa Barrera Mason Gooding Jenna Ortega Tyl...
...,...,...,...,...,...,...,...
312,Kasi Lemmons,Naomi Ackie,Stanley Tucci,Ashton Sanders,Music History Drama,whitney houston: i wanna dance with somebody,Naomi Ackie Stanley Tucci Ashton Sanders Kasi ...
313,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Thriller Crime Horror Mystery,the pale blue eye,Christian Bale Harry Melling Gillian Anderson ...
314,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,Drama,women talking,Rooney Mara Claire Foy Jessie Buckley Sarah Po...
315,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Comedy Drama,a man called otto,Tom Hanks Mariana Treviño Rachel Keller Marc F...


In [101]:
old = pd.read_csv(r'D:\Data Science Pojects\Netflix Recommendation System\Datasets\data 2020_2021\last_data.csv')

old.sample(2)

,director_name,actor_1_name,actor_2_name,actor_3_name,movie_title,genres,comb
1923,Giuseppe Tornatore,Jim Sturgess,Sean Buchanan,Liya Kebede,the best offer,Crime Drama Mystery Romance,Jim Sturgess Sean Buchanan Liya Kebede Giusepp...
1168,Nick Cassavetes,Don Johnson,Kate Upton,David Thornton,the other woman,Comedy Romance,Don Johnson Kate Upton David Thornton Nick Cas...


In [102]:
final = pd.concat([old,new_df22],ignore_index=True)
final

,director_name,actor_1_name,actor_2_name,actor_3_name,movie_title,genres,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,avatar,Action Adventure Fantasy Sci-Fi,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,pirates of the caribbean: at world's end,Action Adventure Fantasy,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,spectre,Action Adventure Thriller,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,the dark knight rises,Action Thriller,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,star wars: episode vii - the force awakens ...,Documentary,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
7143,Kasi Lemmons,Naomi Ackie,Stanley Tucci,Ashton Sanders,whitney houston: i wanna dance with somebody,Music History Drama,Naomi Ackie Stanley Tucci Ashton Sanders Kasi ...
7144,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,the pale blue eye,Thriller Crime Horror Mystery,Christian Bale Harry Melling Gillian Anderson ...
7145,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,women talking,Drama,Rooney Mara Claire Foy Jessie Buckley Sarah Po...
7146,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,a man called otto,Comedy Drama,Tom Hanks Mariana Treviño Rachel Keller Marc F...


In [103]:
final.isna().sum()

director_name    0
actor_1_name     5
actor_2_name     0
actor_3_name     0
movie_title      0
genres           0
comb             0
dtype: int64

In [104]:
final.dropna(inplace=True)

In [106]:
final[final['actor_2_name']=='unknown']

,director_name,actor_1_name,actor_2_name,actor_3_name,movie_title,genres,comb
255,Christopher Barnard,Mathew Buck,unknown,unknown,"10,000 b.c.",Comedy,Mathew Buck unknown unknown Christopher Barnar...
2057,unknown,Chris Harrison,unknown,unknown,the bachelor,Game-Show Reality-TV Romance,Chris Harrison unknown unknown unknown Game-Sh...
2153,unknown,Pablo Sevilla,unknown,unknown,yu-gi-oh! duel monsters,Action Adventure Animation Family Fantasy,Pablo Sevilla unknown unknown unknown Action A...
3231,J.C. Chandor,Robert Redford,unknown,unknown,all is lost,Action Adventure Drama,Robert Redford unknown unknown J.C. Chandor Ac...
4380,Léa Pool,unknown,unknown,unknown,"pink ribbons, inc.",Documentary,unknown unknown unknown Léa Pool Documentary
4397,Harry Gantz,unknown,unknown,unknown,sex with strangers,Documentary Drama,unknown unknown unknown Harry Gantz Documentar...
4598,U. Roberto Romano,unknown,unknown,unknown,the harvest/la cosecha,Documentary,unknown unknown unknown U. Roberto Romano Docu...
4693,Mor Loushy,Amos Oz,unknown,unknown,censored voices,Documentary History,Amos Oz unknown unknown Mor Loushy Documentary...
4715,Pan Nalin,unknown,unknown,unknown,ayurveda: art of being,Documentary,unknown unknown unknown Pan Nalin Documentary
4823,Amal Al-Agroobi,unknown,unknown,unknown,the brain that sings,Documentary Family,unknown unknown unknown Amal Al-Agroobi Docume...


In [107]:
final.drop_duplicates(keep='last',inplace=True)

final

,director_name,actor_1_name,actor_2_name,actor_3_name,movie_title,genres,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,avatar,Action Adventure Fantasy Sci-Fi,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,pirates of the caribbean: at world's end,Action Adventure Fantasy,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,spectre,Action Adventure Thriller,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,the dark knight rises,Action Thriller,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,star wars: episode vii - the force awakens ...,Documentary,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
7143,Kasi Lemmons,Naomi Ackie,Stanley Tucci,Ashton Sanders,whitney houston: i wanna dance with somebody,Music History Drama,Naomi Ackie Stanley Tucci Ashton Sanders Kasi ...
7144,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,the pale blue eye,Thriller Crime Horror Mystery,Christian Bale Harry Melling Gillian Anderson ...
7145,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,women talking,Drama,Rooney Mara Claire Foy Jessie Buckley Sarah Po...
7146,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,a man called otto,Comedy Drama,Tom Hanks Mariana Treviño Rachel Keller Marc F...


In [109]:
final.to_csv('last1_data.csv',index=False)